## 🐍 Python 텍스트 분석: Embedding API 정복하기 (Part 5)

이 학습자료에서는 `OpenAI`, `OpenRouter`, `Hugging Face` 등 주요 AI 플랫폼들의 임베딩 API를 사용하여 텍스트를 벡터로 변환하는 방법을 다룹니다. 

각 서비스의 특징과 사용법을 익히고, 이를 손쉽게 전환하며 사용하는 실용적인 코드 패턴까지 학습합니다.

-----

## 0\. 준비 : 공통 환경 세팅

```bash
# 1) Python 가상환경 준비 (여러분들이 사용하시는 로컬 환경으로 설정 요망)
python -m venv .venv && source .venv/bin/activate

# 2) 공통 패키지
pip install --upgrade openai tiktoken numpy requests sentence-transformers
```

환경 변수로 API 키를 보관하는 것이 가장 안전합니다.

### 환경 변수 설정 방법

**1: .env 파일 사용 (권장)**

프로젝트 루트 디렉토리에 `.env` 파일을 생성하고 API 키들을 저장합니다:

```bash
# .env 파일 생성
OPENAI_API_KEY=sk-...
OPENROUTER_API_KEY=or-...
HUGGINGFACE_API_KEY=hf-...
```

**2: python-dotenv 패키지 사용**

```python
from dotenv import load_dotenv
load_dotenv() # .env 파일의 경로가 다를경우 직접 상대경로 또는 절대 경로를 입력해주어야합니다. load_dotenv(path="경로")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
```


-----



## 1\. OpenAI 임베딩 API 사용법

| 최신 모델 | 최대 컨텍스트 | 가격 (입력 + 출력 합산) |
| :--- | :--- | :--- |
| `text-embedding-3-small` | 8,191 tokens | **$0.02 / 1M tokens** |
| `text-embedding-3-large` | 8,191 tokens | $0.13 / 1M tokens |

> `text-embedding-3-small`는 이전 세대(`ada-002`) 대비 5배 더 저렴하고 품질도 높습니다.

### 1-1. 계정·키 만들기

1.  [https://platform.openai.com](https://platform.openai.com) 에 가입합니다.
2.  “API Keys” 메뉴에서 “Create new secret key”를 클릭합니다.
3.  키를 복사해 `OPENAI_API_KEY` 환경 변수에 저장합니다.

### 1-2. 간단 예제

In [4]:
!pip install openai dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# API 키는 환경 변수(`OPENAI_API_KEY`)에서 자동으로 로드됩니다.
client = OpenAI()
text_list = ["인공지능의 미래는 밝다.", "오늘 점심은 무엇을 먹을까?"]

response = client.embeddings.create(
    model="text-embedding-3-small",
    input=text_list
)

vectors = [obj.embedding for obj in response.data]
print(f"벡터 차원: {len(vectors[0])}")
print(f"첫 번째 벡터 (앞 5개 차원): {vectors[0][:5]}")

벡터 차원: 1536
첫 번째 벡터 (앞 5개 차원): [-0.0006629062700085342, 0.006121872924268246, -0.022473985329270363, 0.0017351238057017326, 0.013250499032437801]


In [31]:
# numpy 배열이면 supabase가 인식 x
vectors

[[-0.0006629062700085342,
  0.006121872924268246,
  -0.022473985329270363,
  0.0017351238057017326,
  0.013250499032437801,
  -0.030019549652934074,
  -0.004240566398948431,
  0.04962581396102905,
  0.002905855653807521,
  0.022412968799471855,
  -0.009106540121138096,
  -0.02947041019797325,
  0.0055117192678153515,
  -0.04722587391734123,
  -0.02743656560778618,
  -0.01879272609949112,
  -0.03642616048455238,
  -0.04122603312134743,
  0.03898880258202553,
  -0.07004561275243759,
  -0.026358628645539284,
  0.02544339746236801,
  0.015680942684412003,
  -0.013219990767538548,
  0.013494559563696384,
  -0.007235402707010508,
  0.026500998064875603,
  0.0139826824888587,
  0.011694607324898243,
  0.004471916239708662,
  0.019514741376042366,
  -0.024121398106217384,
  -0.025463735684752464,
  -0.05544260889291763,
  0.040839601308107376,
  0.04742925986647606,
  0.0052778273820877075,
  0.016891080886125565,
  -0.014043698087334633,
  -0.006762533914297819,
  0.002571542514488101,
  -0.0

### 1-3. 비용 감 잡기

  - **문서 1,000개** (각 500 토큰) ≈ 500,000 토큰 ≈ **$0.01** (약 13원)
  - **Chunking**: 문서를 512 토큰 이하로 잘라 처리하면 품질과 비용을 최적화할 수 있습니다.
  - **캐싱**: 동일한 문장은 한 번만 임베딩하여 결과를 저장하고 재사용하는 것이 비용 절감에 필수적입니다.

-----

## 2\. OpenRouter (무료/저요금) 임베딩 API 사용법

OpenRouter는 400개 이상의 다양한 모델을 OpenAI API와 호환되는 형식으로 제공하는 프록시 서비스입니다. 여러 모델이 무료 또는 매우 저렴하게 제공되어 개발, 연구, 학습용으로 인기가 높습니다.

### 2-1. 계정·키 만들기

1.  [https://openrouter.ai](https://openrouter.ai) 에 가입합니다.
2.  **Dashboard → API Keys** 메뉴에서 키를 발급받습니다.
3.  발급받은 키를 `OPENROUTER_API_KEY` 환경 변수에 저장합니다.

### 2-2. 무료 임베딩 모델 예시

| 모델 ID (OpenRouter) | 가격 | 특징 |
| :--- | :--- | :--- |
| `google/gemma-2-9b-it:free` | $0 / token | Google의 최신 모델, 다국어 지원 |
| `mistralai/mistral-7b-instruct:free` | $0 / token | 작고 빠른 모델, 준수한 성능 |
| `jinaai/jina-embeddings-v2-base-en`| $0.01 / 1M tokens| 임베딩 전용 모델, 영어 특화 |

> **참고**: 무료 모델은 분당 토큰 처리량(TPM)과 동시 요청 수에 제한이 있습니다.

### 2-3. 최소 예제 (OpenAI SDK 그대로)

OpenAI SDK 코드에서 `base_url`과 `model` ID만 변경하면 바로 사용할 수 있습니다.

In [ ]:
# - openrouter 무료 임베딩 X
# - 채팅 api 활용↓↓
# - 오늘은 openai 사용

In [11]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "인공지능의 미래에 대해 간단히 설명해주세요."}
]

response = client.chat.completions.create(
    model="meta-llama/llama-3.1-8b-instruct:free", # OpenRouter에서 제공하는 모델 ID
    messages=messages,
    max_tokens=200,
    temperature=0.7
)

print(f"응답: {response.choices[0].message.content}")
print(f"사용된 토큰 수: {response.usage.total_tokens}")

응답: 인공지능(AI)의 미래는 미래가 밝습니다. 따라서, 사람들은 인공지능(AI)과 컴퓨터 안에 인간의 지성과 감정을 심는 방식에 대해 연구하고 개발하고 있습니다. 

인공지능(AI)은 가능성은 끝이 없는 것처럼 보입니다. 예를 들어, 인공지능(AI)은 다음과 같이 많은 일을 나눌 수 있습니다.

1. 사물인식: 인공지능(AI)은 이미 카메라 및 센서를 통해 물체 인식, 위치 인식 및 객체 인식을 할 수 있습니다. 

2. 자연어 처리: 인공지능(AI)은 이미 사람과 같은 자연어 처리를 할 수 있습니다. 

3. 머신 러닝: 인공지능(AI)은 이미 기계 학습 기반의 알고리즘을 통해 사람들보다 더 잘 학습할 수 있습니다.

따라서, 인공지능(AI)의 미래
사용된 토큰 수: 246


### 2-4. 품질 및 제한 팁

  * **속도**: 무료 모델은 응답이 느릴 수 있으므로, 대량 작업 시에는 배치 처리와 `time.sleep()`을 이용한 간격 조절이 필요합니다.
  * **품질**: 모델마다 성능 편차가 크므로, 작은 데이터셋으로 사전 테스트 후 사용할 모델을 결정하는 것이 좋습니다.
  * **가용성**: 무료 모델은 서버 용량이 가득 차면 에러를 반환할 수 있습니다. `try-except`와 재시도 로직을 구현하면 안정성을 높일 수 있습니다.

-----

## 3\. Hugging Face 임베딩 API 사용법

Hugging Face는 전 세계 AI 모델의 허브로, **Inference API**를 통해 수많은 오픈소스 모델을 서버리스 형태로 사용할 수 있게 지원합니다. 특히 한국어 특화 임베딩 모델을 무료 티어에서 테스트하기에 매우 유용합니다.

### 3-1. 계정·키 만들기

1.  [https://huggingface.co](https://huggingface.co) 에 가입합니다.
2.  **Profile → Settings → Access Tokens** 메뉴로 이동합니다.
3.  `New token` 버튼을 눌러 'read' 권한의 키를 발급하고, 이 값을 `HUGGINGFACE_API_KEY` 환경 변수에 저장합니다.

### 3-2. 추천 한국어 임베딩 모델

| 모델 ID (Hugging Face) | 특징 |
| :--- | :--- |
| `jhgan/ko-sroberta-multitask` | 다양한 한국어 태스크로 학습되어 범용성이 높음 (SBERT 기반) |
| `snunlp/KR-SBERT-V40K-klueNLI-augSTS` | 한국어 문장 유사도(STS) 측정에 특화된 모델 |
| `google-bert/bert-base-multilingual-cased` | 다국어 모델로, 여러 언어가 섞인 환경에서 유용 |

### 3-3. 최소 예제 (`requests` 라이브러리)

Hugging Face Inference API는 `requests` 라이브러리를 통해 직접 호출할 수 있습니다.

In [ ]:
import os
import requests
import numpy as np

# 사용할 모델의 API URL
API_URL = "https://api-inference.huggingface.co/models/jhgan/ko-sroberta-multitask"
headers = {"Authorization": f"Bearer {os.getenv('HUGGINGFACE_API_KEY')}"}

def get_hf_embedding(text_list: list[str]) -> np.ndarray | None:
    try:
        response = requests.post(API_URL, headers=headers, json={"inputs": text_list})
        response.raise_for_status()
        # Hugging Face API는 벡터 리스트를 직접 반환합니다.
        return np.array(response.json())
    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 에러가 발생했습니다: {e}")
        return None

# API 호출
text_list = ["인공지능의 미래는 밝다.", "오늘 점심은 무엇을 먹을까?"]
vectors = get_hf_embedding(text_list)

if vectors is not None:
    print(f"벡터 차원: {len(vectors[0])}")
    print(f"첫 번째 벡터 (앞 5개 차원): {vectors[0][:5]}")

### 3-4. (심화) `sentence-transformers` 라이브러리 활용

`sentence-transformers` 라이브러리를 사용하면 Hugging Face의 모델을 더 간편하게 다룰 수 있습니다. API 호출 대신 모델을 로컬에 다운로드하여 실행하는 방식이지만, 코드가 매우 직관적입니다.

In [ ]:
from sentence_transformers import SentenceTransformer

# 모델을 로컬에 다운로드하고 초기화합니다. (최초 실행 시 시간 소요)
model = SentenceTransformer('jhgan/ko-sroberta-multitask') # 한국어 임베딩을 잘 해주는 모델

text_list = ["인공지능의 미래는 밝다.", "오늘 점심은 무엇을 먹을까?"]
vectors = model.encode(text_list)

print(f"벡터 차원: {vectors.shape[1]}")
print(f"첫 번째 벡터 (앞 5개 차원): {vectors[0][:5]}")

c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--jhgan--ko-sroberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

벡터 차원: 768
첫 번째 벡터 (앞 5개 차원): [-0.3747718   0.62013036  0.13008434  0.25405574 -0.19390945]


In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1.5')   # Do NOT set trust_remote_code
model.max_seq_length = 512

embeddings = model.encode(
    sentences, 
    normalize_embeddings=True,
    batch_size=256, 
    show_progress_bar=True
    )
print(embeddings)

c:\Users\Admin\workspace\analysis_prep\class\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--HIT-TMG--KaLM-embedding-multilingual-mini-instruct-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.35it/s]

[[-0.05421966 -0.0076393   0.03664602 ... -0.03581953  0.00133925
  -0.01778297]
 [-0.03844889  0.00202709  0.04016364 ... -0.03577899 -0.00159532
   0.00557057]]


-----

## 4\. 여러 환경을 자유롭게 전환하는 패턴

각기 다른 API를 하나의 인터페이스로 통일하면 코드 변경을 최소화하며 서비스를 전환할 수 있습니다.

In [ ]:
import os
from openai import OpenAI
import requests
import numpy as np

class EmbeddingService:
    def __init__(self, provider: str = "openai", model: str = None):
        self.provider = provider

        if provider == "openai":
            self.client = OpenAI()
            self.model = model or "text-embedding-3-small"
        elif provider == "openrouter":
            self.client = OpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=os.getenv("OPENROUTER_API_KEY"),
            )
            self.model = model or "google/gemma-2-9b-it:free"
        elif provider == "huggingface":
            self.model = model or "jhgan/ko-sroberta-multitask"
            self.api_url = f"https://api-inference.huggingface.co/models/{self.model}"
            self.headers = {"Authorization": f"Bearer {os.getenv('HUGGINGFACE_API_KEY')}"}
        else:
            raise ValueError("지원하지 않는 provider입니다: openai, openrouter, huggingface")

    def embed(self, text_list: list[str]) -> list[list[float]] | None:
        print(f"--- {self.provider}({self.model}) 모델로 임베딩 수행 ---")
        if self.provider in ["openai", "openrouter"]:
            response = self.client.embeddings.create(model=self.model, input=text_list)
            return [obj.embedding for obj in response.data]

        elif self.provider == "huggingface":
            try:
                response = requests.post(self.api_url, headers=self.headers, json={"inputs": text_list})
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                print(f"API 요청 오류: {e}")
                return None

# --- 사용 예시 ---
# service = EmbeddingService(provider="openai")
# service = EmbeddingService(provider="openrouter", model="mistralai/mistral-7b-instruct:free")
service = EmbeddingService(provider="huggingface")

vectors = service.embed(["이것은 테스트 문장입니다."])
if vectors:
    print("임베딩 성공!")

이처럼 \*\*클래스 기반의 래퍼(Wrapper)\*\*를 만들면, `provider` 이름만 바꿔서 일관된 방식으로 모든 서비스를 사용할 수 있습니다.

-----

## 5\. 후속 활용 : 벡터 DB에 저장하기

생성된 임베딩 벡터는 \*\*벡터 데이터베이스(Vector DB)\*\*에 저장하여 의미 기반 검색, 추천 시스템, RAG(검색 증강 생성) 파이프라인 등에 활용할 수 있습니다.

### Supabase를 Vector DB로 활용하기

Supabase는 PostgreSQL 기반의 오픈소스 백엔드 서비스로, **pgvector** 확장을 통해 벡터 데이터베이스 기능을 제공합니다. 임베딩 벡터를 저장하고 의미 기반 검색을 수행할 수 있는 강력한 플랫폼입니다.

#### 주요 특징:
- **pgvector 확장**: 벡터 유사도 검색 지원
- **SQL 기반**: 익숙한 SQL 쿼리로 벡터 검색 가능
- **실시간 API**: REST API와 실시간 구독 기능
- **무료 티어**: 개발 및 학습용으로 충분한 무료 사용량 제공

#### 준비 절차

##### 1. pgvector 확장 설치 (한 번만 실행, 여러분은 이미 설치했어요 지난 시간에)

```sql
-- Postgres 15 이상 / Supabase 기본 버전
create extension if not exists vector cascade;
```

`pgvector` 확장이 활성화되면 **vector**, **halfvec**(16-bit 부동소수형), **svec**(sparse) 자료형과 K-NN 인덱스(HNSW, IVFFLAT 등)를 쓸 수 있습니다. ([supabase.com][1])

---

##### 2. 테이블 생성
   
지난번에 생성했던 테이블이 있다면 콘솔에서 삭제하시고 진행해주세요

```sql
create table if not exists public.embeddings (
  id          bigserial primary key,
  content     text,                       -- 원본 문장 or 문서 요약
  embedding   vector(1536),               -- OpenAI text-embedding-3-small / ada-002 크기
  created_at  timestamptz default now()
);
```

> 🔎 **vector(차원)** 은 모델 출력 차원과 반드시 일치해야 합니다. (text-embedding-3-small 및 text-embedding-ada-002 ⇒ 1536차원)


---

##### 3. K-NN 인덱스 추가(선택)

빈도가 많은 검색·유사도 비교가 필요하다면 다음 중 하나를 추가하세요.

```sql
-- a. Cosine 유사도 IVFFLAT (빠르지만 삽입 전에 REINDEX 필요)
create index if not exists embeddings_embedding_cos_ivfflat
  on public.embeddings using ivfflat (embedding vector_cosine_ops)
  with (lists = 100);       -- 리스트 수는 데이터 규모에 맞춰 조정

-- b. HNSW(삽입·검색 동시성 ↑, 최신 PostgreSQL 필요)
-- create index ... using hnsw (embedding vector_l2_ops) with (m = 16, ef_construction = 64);
```

인덱스를 만들기 전에 `ANALYZE embeddings;`를, IVFFLAT 인덱스 후에는 `REINDEX`를 권장합니다.

---

##### 4. RLS(행 수준 보안) 설정 (어려운 부분이 여러분은 그냥 쿼리만 실행하세요)

Supabase 프로젝트는 기본적으로 **RLS가 켜져** 있습니다. 익명 키(anon key)로 쓰기·읽기를 허용하려면 최소 두 개의 정책이 필요합니다.

```sql
alter table public.embeddings enable row level security;

-- INSERT 허용
create policy "Allow insert for anon" on public.embeddings
  for insert
  to anon
  using (true)          -- 조건을 걸어야 한다면 여기서 필터

-- SELECT 허용
create policy "Allow read for anon" on public.embeddings
  for select
  to anon
  using (true);
```

---

##### 5. 파이썬 클라이언트 사용 팁

```python
# 벡터는 list[float] 형태면 자동 캐스팅됩니다.
vectors = [[0.01, 0.23, ... , 0.45]]  # 1536-dim list
text_list = ["이것은 테스트 문장입니다."]

save_embeddings_to_supabase(vectors, text_list)
```

* **환경 변수**로 `SUPABASE_URL`과 `SUPABASE_ANON_KEY`를 관리해 키 노출을 방지하세요.
* 수천 \~ 수만 건 이상을 넣을 때는 `insert().execute()` 대신 `upsert()` 또는 `copy_from()`(psycopg) 방식을 고려하면 훨씬 빠릅니다.

---

In [19]:
!pip install supabase

  Using cached gotrue-2.12.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached postgrest-1.0.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached realtime-2.4.3-py3-none-any.whl.metadata (6.7 kB)
  Using cached storage3-0.11.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached supafunc-0.9.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached pytest_mock-3.14.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached StrEnum-0.4.15-py3-none-any.whl.metadata (5.3 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 4.1 MB/s eta 0:00:00
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.me


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()
# load_dotenv("C:../경로") - False로 뜨면 직접 경로 넣기

True

In [27]:
import numpy as np
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

# Supabase 클라이언트 설정
url = os.getenv("SUPABASE_URL") # .env 파일에 저장된 환경 변수 사용
key = os.getenv("SUPABASE_ANON_KEY") # .env 파일에 저장된 환경 변수 사용
supabase: Client = create_client(url, key)

# 임베딩 벡터를 Supabase에 저장하는 예제
def save_embeddings_to_supabase(vectors, text_list):
    """
    임베딩 벡터와 텍스트를 Supabase 테이블에 저장
    
    테이블 구조 예시:
    - id: SERIAL PRIMARY KEY
    - content: TEXT
    - embedding: VECTOR(1536) -- 또는 해당 모델의 차원 수
    - created_at: TIMESTAMP
    """
    
    # 데이터 생성 후 insert
    data_to_insert = []
    for i, (text, vector) in enumerate(zip(text_list, vectors)):
        data_to_insert.append({
            "content": text,
            "embedding": vector  # Supabase는 리스트 형태의 벡터를 자동으로 처리
        })

    # 이걸 함수로 만듦
    # supabase.table("embeddings").insert([
    #     {
    #         "content": "안녕하세요",
    #         "embedding": [..., ,...]
    #     },
    #     {
    #         "content": "안녕하세요",
    #         "embedding": [..., ,...]
    #     },
    #     {
    #         "content": "안녕하세요",
    #         "embedding": [..., ,...]
    #     }
    # ]).execute()
    
    try:
        result = supabase.table("embeddings").insert(data_to_insert).execute()
        print(f"성공적으로 {len(data_to_insert)}개의 임베딩을 저장했습니다.")
        return result
    except Exception as e:
        print(f"Supabase 저장 중 오류 발생: {e}")
        return None

In [28]:
len(vectors)

2

In [32]:
# 사용 예시
if vectors:
    save_embeddings_to_supabase(vectors, text_list)

성공적으로 2개의 임베딩을 저장했습니다.
